![picture](https://miro.medium.com/max/1262/1*LBn_F2ALsKUAKK6vbQbJuw.png)

# Fake News Classifier Using LSTM

Dataset Link - https://www.kaggle.com/c/fake-news/data#

#### Dataset : A full training dataset with the following attributes:  

id: unique id for a news article  
title: the title of a news article  
author: author of the news article  
text: the text of the article; could be incomplete  
label: a label that marks the article as potentially unreliable  

### Importing Required Libraries

In [284]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Dropout
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading Dataset

In [285]:
data= pd.read_csv("/content/drive/MyDrive/Deep Learning/train.csv")

In [286]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [287]:
data.shape

(20800, 5)

In [288]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

### Droping Null Values

In [289]:
data=data.dropna()

In [290]:
data.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [291]:
data.shape

(18285, 5)

### Spliting Data Into Independent and Dependent Variable

In [292]:
X=data.drop("label",axis=1)

In [293]:
y=data["label"]

### Defining Vocabulary Size

In [294]:
voc_size=5000

### Onehot Representation

In [295]:
messages=X.copy()

In [296]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [297]:
messages.reset_index(inplace=True)

In [298]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Dataset Preprocessing

In [299]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [300]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [301]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2022, 1656, 3458, 4324, 1764, 3246, 3452, 4599, 1114, 2305],
 [4497, 237, 2516, 2578, 203, 1053, 3196],
 [757, 1520, 4044, 4361],
 [405, 869, 1320, 379, 3038, 650],
 [314, 203, 102, 629, 760, 441, 203, 4453, 4667, 1950],
 [4031,
  1528,
  1824,
  2637,
  3944,
  4538,
  616,
  876,
  1554,
  771,
  903,
  4932,
  4329,
  2057,
  3196],
 [28, 4420, 683, 2861, 4196, 1226, 4947, 1418, 2229, 2187, 4394],
 [1276, 442, 2225, 2376, 4932, 2060, 4538, 72, 2229, 2187, 4394],
 [2301, 3062, 2018, 2305, 228, 520, 22, 681, 4538, 3358],
 [3231, 1326, 3726, 4098, 169, 2151, 3875, 4565],
 [940, 286, 2539, 4381, 3032, 4294, 3275, 212, 1303, 2170, 4165],
 [379, 4199, 1764, 520, 4538, 4932],
 [4788, 4084, 2485, 602, 710, 3765, 418, 2358, 39],
 [248, 1694, 1915, 4368, 2442, 3649, 1829, 2229, 2187, 4394],
 [1109, 2198, 2271, 3869, 2868, 2229, 2187, 4394],
 [907, 1214, 1360, 2246, 1349, 495, 2460, 4647, 3393, 4246],
 [1829, 1357, 237],
 [2679, 2016, 4766, 702, 4538, 1496, 2701, 3196],
 [135, 4109, 2516, 48

### Embedding Representation

In [302]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4599 1114 2305]
 [   0    0    0 ...  203 1053 3196]
 [   0    0    0 ... 1520 4044 4361]
 ...
 [   0    0    0 ... 2229 2187 4394]
 [   0    0    0 ... 3494 2838 1200]
 [   0    0    0 ... 2938 3553 2711]]


In [303]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2022,
       1656, 3458, 4324, 1764, 3246, 3452, 4599, 1114, 2305], dtype=int32)

### Creating Model Without Dropout Layer

In [304]:
adam = keras.optimizers.Adam(learning_rate=0.00001)
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(150))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])
print(model.summary())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 20, 40)            200000    
                                                                 
 lstm_16 (LSTM)              (None, 150)               114600    
                                                                 
 dense_16 (Dense)            (None, 1)                 151       
                                                                 
Total params: 314,751
Trainable params: 314,751
Non-trainable params: 0
_________________________________________________________________
None


In [305]:
len(embedded_docs),y.shape

(18285, (18285,))

In [306]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [307]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [308]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=75)

### Model Training

In [309]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
200/200 [==============================] - 4s 11ms/step - loss: 0.6897 - accuracy: 0.6109 - val_loss: 0.6872 - val_accuracy: 0.5591
Epoch 2/50
200/200 [==============================] - 2s 10ms/step - loss: 0.6836 - accuracy: 0.5709 - val_loss: 0.6805 - val_accuracy: 0.5578
Epoch 3/50
200/200 [==============================] - 2s 10ms/step - loss: 0.6742 - accuracy: 0.5704 - val_loss: 0.6693 - val_accuracy: 0.5578
Epoch 4/50
200/200 [==============================] - 1s 7ms/step - loss: 0.6583 - accuracy: 0.5731 - val_loss: 0.6504 - val_accuracy: 0.5667
Epoch 5/50
200/200 [==============================] - 2s 8ms/step - loss: 0.6341 - accuracy: 0.6238 - val_loss: 0.6264 - val_accuracy: 0.6185
Epoch 6/50
200/200 [==============================] - 1s 7ms/step - loss: 0.6082 - accuracy: 0.7041 - val_loss: 0.6044 - val_accuracy: 0.7331
Epoch 7/50
200/200 [==============================] - 2s 8ms/step - loss: 0.5859 - accuracy: 0.7584 - val_loss: 0.5845 - val_accuracy: 0.7765
Epo

### Performance Metrics And Accuracy

In [310]:
y_pred=model.predict(X_test)

172/172 [==============================] - 1s 2ms/step


In [311]:
y_pred=np.round(abs(y_pred))

In [312]:
accuracy_score(y_test, y_pred)

0.9015676266861101

In [313]:
confusion_matrix(y_test,y_pred)


array([[2777,  283],
       [ 257, 2169]])

In [314]:
print("Accuracy = ",accuracy_score(y_test, y_pred)*100)

Accuracy =  90.15676266861101


### Creating Model With Dropout Layer

In [315]:
adam = keras.optimizers.Adam(learning_rate=0.00001)
embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Dropout(0.3))
model2.add(LSTM(150))
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])

In [316]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
200/200 [==============================] - 5s 15ms/step - loss: 0.6910 - accuracy: 0.5744 - val_loss: 0.6891 - val_accuracy: 0.5578
Epoch 2/50
200/200 [==============================] - 1s 7ms/step - loss: 0.6862 - accuracy: 0.5707 - val_loss: 0.6840 - val_accuracy: 0.5578
Epoch 3/50
200/200 [==============================] - 1s 7ms/step - loss: 0.6794 - accuracy: 0.5706 - val_loss: 0.6763 - val_accuracy: 0.5578
Epoch 4/50
200/200 [==============================] - 1s 7ms/step - loss: 0.6685 - accuracy: 0.5712 - val_loss: 0.6634 - val_accuracy: 0.5578
Epoch 5/50
200/200 [==============================] - 1s 7ms/step - loss: 0.6513 - accuracy: 0.5973 - val_loss: 0.6447 - val_accuracy: 0.6258
Epoch 6/50
200/200 [==============================] - 2s 8ms/step - loss: 0.6317 - accuracy: 0.6499 - val_loss: 0.6270 - val_accuracy: 0.6938
Epoch 7/50
200/200 [==============================] - 1s 7ms/step - loss: 0.6145 - accuracy: 0.7065 - val_loss: 0.6118 - val_accuracy: 0.7457
Epoch

### Performance Metrics And Accuracy

In [317]:
y_pred2=model2.predict(X_test)

172/172 [==============================] - 1s 3ms/step


In [318]:
y_pred2=np.round(abs(y_pred2))

In [319]:
accuracy_score(y_test, y_pred2)

0.9041195771053591

In [320]:
confusion_matrix(y_test,y_pred2)

array([[2758,  302],
       [ 224, 2202]])

In [321]:
print("Accuracy =",accuracy_score(y_test, y_pred2)*100)

Accuracy = 90.4119577105359
